git checkout master   
git branch main master -f    
git checkout main  
git push origin main -f 


In [2]:
import pandas as pd 
import os 
pd.set_option('display.max_columns', None)  # Tüm sütunları gösterir
pd.set_option('display.max_colwidth', None) # Sütun genişliklerini sınırlandırmadan gösterir

In [ ]:
#fbrefscrape ile scrape edilen maç linkleri df inden istediğimiz linkleri ayıklayıp filtered_week_match_links.txt olarak scrape e gönderir.
df = pd.read_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv', parse_dates=['date'])
week_match_links = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_match_links.json')  
all_team_names = list(set(list(df.home_team.unique()) + list(df.away_team.unique()))) # -> datada bulunan tum takimlar. linklerden istediklerimizi çekmek için lazım olcak.
eklenmesi_gereken_takimlar = ['fenerbahce','adana-demirspor','besiktas','eyupspor'] #-> link kısmı turkce karakter icermiyor df iceriyor
all_team_names = all_team_names + eklenmesi_gereken_takimlar 
filtered_week_match_links = week_match_links[week_match_links['url'].str.contains('|'.join(all_team_names), case=False)]
filtered_week_match_links = list(filtered_week_match_links.url.values)
directory = '/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/sofa_temiz_urls'
file_path = os.path.join(directory, f'filtered_week_match_links.txt')
with open(file_path, 'w') as file:
    for url in filtered_week_match_links:
        file.write(url + '\n')

In [4]:
# BURAYI BASMADAN ONCE GEREKLI VERILERDE EKSIK VAR MI KONTROL ET DUZELT (EKSIK SARI VE KIRMIZI KART SORUN DEGIL)
df = pd.read_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv', parse_dates=['date'])
week_matches = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_matches.json',convert_dates= False)
week_matches_first_half = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_matches_first_half.json')
merged_week_matches = pd.merge(week_matches,week_matches_first_half, on= 'match_link', how='left')
merged_week_matches = merged_week_matches[merged_week_matches.Organizasyon.isin(df.organizasyon.unique())] #-> istediğim organizsayon isimlerni çeker.
merged_week_matches = merged_week_matches[merged_week_matches.ulke.isin(df.ulke.unique())] #-> ulkeler icin tekrar bi ayikladim bazen bir organizasyon iki ulkede olabiliyor.
# kart verileri eksikse 0 oldugundan eksik oluyorlar onları 0 yaptım 
merged_week_matches['Yellow cards_home'] = merged_week_matches['Yellow cards_home'].fillna(0).astype(int)
merged_week_matches['Yellow cards_away'] = merged_week_matches['Yellow cards_away'].fillna(0).astype(int)
merged_week_matches['Red cards_home'] = merged_week_matches['Red cards_home'].fillna(0).astype(int)
merged_week_matches['Red cards_away'] = merged_week_matches['Red cards_away'].fillna(0).astype(int)
merged_week_matches['Yellow cards_home_firsthalf'] = merged_week_matches['Yellow cards_home_firsthalf'].fillna(0).astype(int)
merged_week_matches['Yellow cards_away_firsthalf'] = merged_week_matches['Yellow cards_away_firsthalf'].fillna(0).astype(int)
merged_week_matches['Red cards_home_firsthalf'] = merged_week_matches['Red cards_home_firsthalf'].fillna(0).astype(int)
merged_week_matches['Red cards_away_firsthalf'] = merged_week_matches['Red cards_away_firsthalf'].fillna(0).astype(int)
merged_week_matches['date'] = pd.to_datetime(merged_week_matches['date'], format='%d-%m-%Y') #--> date todatetime a cevrildi
merged_week_matches.columns = merged_week_matches.columns.str.lower().str.replace(' ', '_') #--> sutun isimleri kucuk harf oldu ve bosluklar alt tire ile degisti.
# season_round ve season_period sutunlarını olusturur.
df_leagues = merged_week_matches[~(merged_week_matches.ulke == 'Europe')]
df_leagues.loc[ : , 'season_round'] = df_leagues.org_detay.apply(lambda x: x[-2:]).astype(int)
df_leagues = df_leagues[['match_link','season_round']]
df_leagues.loc[df_leagues.season_round <= 9 , 'season_period'] = 1
df_leagues.loc[(df_leagues.season_round > 9) & (df_leagues.season_round <= 18)  , 'season_period'] = 2
df_leagues.loc[(df_leagues.season_round > 18) & (df_leagues.season_round <= 27)  , 'season_period'] = 3
df_leagues.loc[(df_leagues.season_round >= 28)  , 'season_period'] = 4
merged_week_matches = pd.merge(merged_week_matches,df_leagues, on= 'match_link', how= 'left')
merged_week_matches = merged_week_matches.drop_duplicates()
merged_week_matches = merged_week_matches.fillna(0)
merged_week_matches.season_period = merged_week_matches.season_period.astype(int)
merged_week_matches.season_round = merged_week_matches.season_round.astype(int)
merged_week_matches.reset_index(drop= True, inplace= True)
# ikincyi yari sutunları olusturur.
merged_week_matches['total_shots_away_secondhalf'] = merged_week_matches['total_shots_away'] - merged_week_matches['total_shots_away_firsthalf']
merged_week_matches['total_shots_home_secondhalf'] = merged_week_matches['total_shots_home'] - merged_week_matches['total_shots_home_firsthalf']
merged_week_matches['shots_on_target_home_secondhalf'] = merged_week_matches['shots_on_target_home'] - merged_week_matches['shots_on_target_home_firsthalf']
merged_week_matches['shots_on_target_away_secondhalf'] = merged_week_matches['shots_on_target_away'] - merged_week_matches['shots_on_target_away_firsthalf']
merged_week_matches['shots_inside_box_home_secondhalf'] = merged_week_matches['shots_inside_box_home'] - merged_week_matches['shots_inside_box_home_firsthalf']
merged_week_matches['shots_inside_box_away_secondhalf'] = merged_week_matches['shots_inside_box_away'] - merged_week_matches['shots_inside_box_away_firsthalf']
merged_week_matches['shots_outside_box_home_secondhalf'] = merged_week_matches['shots_outside_box_home'] - merged_week_matches['shots_outside_box_home_firsthalf']
merged_week_matches['shots_outside_box_away_secondhalf'] = merged_week_matches['shots_outside_box_away'] - merged_week_matches['shots_outside_box_away_firsthalf']
merged_week_matches['corner_kicks_home_secondhalf'] = merged_week_matches['corner_kicks_home'] - merged_week_matches['corner_kicks_home_firsthalf']
merged_week_matches['corner_kicks_away_secondhalf'] = merged_week_matches['corner_kicks_away'] - merged_week_matches['corner_kicks_away_firsthalf']
merged_week_matches['passes_home_secondhalf'] = merged_week_matches['passes_home'] - merged_week_matches['passes_home_firsthalf']
merged_week_matches['passes_away_secondhalf'] = merged_week_matches['passes_away'] - merged_week_matches['passes_away_firsthalf']
merged_week_matches['yellow_cards_home_secondhalf'] = merged_week_matches['yellow_cards_home'] - merged_week_matches['yellow_cards_home_firsthalf']
merged_week_matches['yellow_cards_away_secondhalf'] = merged_week_matches['yellow_cards_away'] - merged_week_matches['yellow_cards_away_firsthalf']
merged_week_matches['red_cards_home_secondhalf'] = merged_week_matches['red_cards_home'] - merged_week_matches['red_cards_home_firsthalf']
merged_week_matches['red_cards_away_secondhalf'] = merged_week_matches['red_cards_away'] - merged_week_matches['red_cards_away_firsthalf']
merged_week_matches['free_kicks_home_secondhalf'] = merged_week_matches['free_kicks_home'] - merged_week_matches['free_kicks_home_firsthalf']
merged_week_matches['free_kicks_away_secondhalf'] = merged_week_matches['free_kicks_away'] - merged_week_matches['free_kicks_away_firsthalf']
#--> 2024-2025 sezonu season 8 olarak alıncak
merged_week_matches['season'] = 8 
# result ve iy_result sütunaları 
merged_week_matches['result'] = merged_week_matches.apply(lambda row: 1 if row['home_goal'] > row['away_goal'] 
                        else 2 if row['home_goal'] < row['away_goal'] 
                        else 0, axis=1)
merged_week_matches['iy_result'] = merged_week_matches.apply(lambda row: 1 if row['home_iy_goal'] > row['away_iy_goal'] 
                        else 2 if row['home_goal'] < row['away_goal'] 
                        else 0, axis=1)
# istedigim sutunları aldım sadece 
merged_week_matches = merged_week_matches[['date', 'hour', 'ulke', 'organizasyon', 'org_detay', 'home_team','home_goal', 'away_goal', 'away_team', 'result', 'home_iy_goal','away_iy_goal', 
'iy_result', 'corner_kicks_home','corner_kicks_home_firsthalf', 'corner_kicks_away','corner_kicks_away_firsthalf', 'total_shots_home','total_shots_home_firsthalf', 'shots_on_target_home',
'shots_on_target_home_firsthalf', 'total_shots_away','total_shots_away_firsthalf', 'shots_on_target_away','shots_on_target_away_firsthalf', 'fouls_home', 'fouls_away','yellow_cards_home', 
'yellow_cards_home_firsthalf', 'yellow_cards_away','yellow_cards_away_firsthalf', 'red_cards_home','red_cards_home_firsthalf', 'red_cards_away','red_cards_away_firsthalf', 'referee', 'red_card_avg',
'yellow_card_avg', 'shots_inside_box_home', 'shots_inside_box_away','shots_outside_box_home', 'shots_outside_box_away', 'free_kicks_home','free_kicks_away', 'ball_possession_home', 'ball_possession_away',
'passes_home', 'passes_away', 'ball_possession_home_firsthalf','ball_possession_away_firsthalf', 'passes_home_firsthalf','passes_away_firsthalf', 'free_kicks_home_firsthalf','free_kicks_away_firsthalf',
'shots_inside_box_home_firsthalf','shots_inside_box_away_firsthalf', 'shots_outside_box_home_firsthalf','shots_outside_box_away_firsthalf', 'total_shots_home_secondhalf','total_shots_away_secondhalf', 
'shots_on_target_home_secondhalf','shots_on_target_away_secondhalf', 'shots_inside_box_home_secondhalf','shots_inside_box_away_secondhalf', 'shots_outside_box_home_secondhalf','corner_kicks_home_secondhalf', 
'corner_kicks_away_secondhalf','passes_home_secondhalf', 'passes_away_secondhalf','free_kicks_home_secondhalf', 'yellow_cards_home_secondhalf','yellow_cards_away_secondhalf', 'red_cards_home_secondhalf',
'red_cards_away_secondhalf', 'free_kicks_away_secondhalf', 'match_link','season', 'season_round', 'season_period']]
# dtypeları ayarlarma 
merged_week_matches.hour = merged_week_matches.hour.str.replace(':','.').astype(float).astype(int)
merged_week_matches.ball_possession_home_firsthalf = merged_week_matches.ball_possession_home_firsthalf.str.replace('%','').astype(int)
merged_week_matches.ball_possession_away_firsthalf = merged_week_matches.ball_possession_away_firsthalf.str.replace('%','').astype(int)
merged_week_matches.ball_possession_away = merged_week_matches.ball_possession_away.str.replace('%','').astype(int)
merged_week_matches.ball_possession_home = merged_week_matches.ball_possession_home.str.replace('%','').astype(int)
# concat islemi
df_concanated = pd.concat([df,merged_week_matches])
df_concanated.sort_values(by= 'date', ascending= False, inplace= True)
# eski df'i yedege alma
df.to_csv('/Users/icy/Desktop/sofa/sofa_all_works/csvler/last_df_yedek.csv',index= False)
# yeni df_i last_df_.csv seklinde atama 
df_concanated.to_csv('/Users/icy/Desktop/sofa/sofa_all_works/csvler/last_df_.csv', index= False)